<a href="https://colab.research.google.com/github/kazzand/TextEmbeddings/blob/master/Doc2vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [39]:
!pip3 install pymorphy2

In [0]:
import pandas as pd
import re
import nltk
import pymorphy2
from gensim.models.doc2vec import TaggedDocument, Doc2Vec
import numpy as np


nltk.download("stopwords")
nltk.download('punkt')

In [4]:
data = pd.read_csv('/content/gdrive/My Drive/lenta-ru-news.csv', header = 0)

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [54]:
data.shape

(800975, 6)

In [0]:
small_data = data[:10000]

In [7]:
small_data.head()

,url,title,text,topic,tags,date
0,https://lenta.ru/news/1914/09/16/hungarnn/,1914. Русские войска вступили в пределы Венгрии,Бои у Сопоцкина и Друскеник закончились отступ...,Библиотека,Первая мировая,1914/09/16
1,https://lenta.ru/news/1914/09/16/lermontov/,1914. Празднование столетия М.Ю. Лермонтова от...,"Министерство народного просвещения, в виду про...",Библиотека,Первая мировая,1914/09/16
2,https://lenta.ru/news/1914/09/17/nesteroff/,1914. Das ist Nesteroff!,"Штабс-капитан П. Н. Нестеров на днях, увидев в...",Библиотека,Первая мировая,1914/09/17
3,https://lenta.ru/news/1914/09/17/bulldogn/,1914. Бульдог-гонец под Льежем,Фотограф-корреспондент Daily Mirror рассказыва...,Библиотека,Первая мировая,1914/09/17
4,https://lenta.ru/news/1914/09/18/zver/,1914. Под Люблином пойман швабский зверь,"Лица, приехавшие в Варшаву из Люблина, передаю...",Библиотека,Первая мировая,1914/09/18


In [0]:
def preprocess_text(data):
  corpus= []
  for text in data['text']:
    reg = re.compile('[^a-zA-Zа-яА-Я. ]')
    corpus.append(reg.sub('', text))
  for i in range(len(corpus)):
    corpus[i] = corpus[i].lower()
  print('lower is fine')
  sentences = []
  for news in corpus:
    sentences.append(nltk.sent_tokenize(news)) 

  print('sentences tokenize is fine')
  morph = pymorphy2.MorphAnalyzer()  
  russian_stopwords = stopwords.words("russian")

  words_bag = []
  for mes in sentences:
    appender = []
    for sentence in mes:
      reg = re.compile('[^a-zA-Zа-яА-Я ]')
      a = (reg.sub('', sentence))
      mas = lemma(a)
      for el in mas:
        appender.append(el)
    appender_sw = [word for word in appender if not word in russian_stopwords]
    words_bag.append(appender_sw)

  print('Everything is fine')
  return words_bag

In [0]:
def lemma(text):
    words = text.split() # разбиваем текст на слова
    res = list()
    for word in words:
        p = morph.parse(word)[0]
        res.append(p.normal_form)

    return res


In [56]:
words_bag = preprocess_text(small_data)

lower is fine
sentences tokenize is fine
Everything is fine


In [0]:
train_data = []
for i, d in enumerate(words_bag):
    train_data.append(TaggedDocument(words = words_bag[i], tags=[str(i)]))



In [58]:
len(train_data)

10000

In [0]:
model = Doc2Vec(vector_size=50, min_count=1, epochs=40)
model.build_vocab(train_data)
model.train(train_data, total_examples=model.corpus_count, epochs=model.epochs)

In [0]:
v1 = model.infer_vector('путин объявил следующую неделю нерабочей из-за коронавируса отдыхайте лечитесь сидите дома'.split(' '))
v2 = model.infer_vector('следующая неделя каникулы об это на заседании объявил путин мы должны изолироваться чтоб избежать заражения коронавирусом'.split(' '))
v3 = model.infer_vector('собака любит кости и играть с мячиком кататься на скейтборде и есть бананы'.split(' '))


In [61]:
def cos(v1,v2):
        return v1 @ v2 / (np.linalg.norm(v1) * np.linalg.norm(v2))
print(cos(v1,v2))
print(cos(v3,v2))
print(cos(v1,v3))

0.70599335
0.019092016
0.23342082
